# Resources

https://matthewrocklin.com/blog/work/2017/09/21/accelerating-geopandas-1
http://www.portailsig.org/content/python-geopandas-ou-le-pandas-spatial

## Pandas SQL doc 

* https://pandas.pydata.org/pandas-docs/stable/io.html#io-sql
* https://pandas.pydata.org/pandas-docs/stable/io.html#engine-connection-examples

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

from sqlalchemy import create_engine
import sqlalchemy

connection_string = 'postgresql://root:@localhost:5432/test_geopandas'
db = create_engine(connection_string, echo=False)
engine = db.connect()

# Check connection
result = engine.execute("SELECT 1")
list(result)

/home/mazzo/Envs/geopandas_dev/lib/python3.5/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


[(1,)]

# Load data

In [32]:
countries = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
cities = gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))
# For consistency
countries = countries.set_index('name').sort_index()
cities = cities.set_index('name').sort_index()

# Add fake dates
cities['year'] = pd.date_range('1900', periods=len(cities), freq='Y' )
countries['year'] = pd.date_range('1900', periods=len(countries), freq='Y' )

# Geo SQL

Using Postgis `GEOMETRY` Schema and shapely figures

## Ingfest a gdf

In [33]:
from geopandas import io

In [34]:
from geoalchemy2 import Geometry, WKTElement
import shapely

In [35]:
# Do not use `geoalchemy.sql.from_shape()`
# https://github.com/geoalchemy/geoalchemy2/issues/132

In [36]:
table_name = 'country'
countries.to_postgis(table_name, engine, if_exists='replace')

In [37]:
retrieved_countries = gpd.read_postgis('country', engine, geom_col='geometry', 
                                       index_col='name', coerce_float=False)
retrieved_countries = retrieved_countries.sort_index()

In [38]:
(countries.dtypes, countries.geom_type.unique(), 
 countries.crs, countries.shape)

(pop_est              float64
 continent             object
 iso_a3                object
 gdp_md_est           float64
 geometry              object
 year          datetime64[ns]
 dtype: object,
 array(['Polygon', 'MultiPolygon'], dtype=object),
 {'init': 'epsg:4326'},
 (177, 6))

In [39]:
(retrieved_countries.dtypes, retrieved_countries.geom_type.unique(), 
 retrieved_countries.crs, retrieved_countries.shape)

(pop_est              float64
 continent             object
 iso_a3                object
 gdp_md_est           float64
 geometry              object
 year          datetime64[ns]
 dtype: object,
 array(['Polygon', 'MultiPolygon'], dtype=object),
 {'init': 'epsg:4326'},
 (177, 6))

### Ingest cities

In [40]:
table_name = 'city'
cities.to_postgis(table_name, engine, if_exists='replace')

In [41]:
# Retrieve
retrieved_cities = gpd.read_postgis(table_name, engine, geom_col='geometry',
                                    index_col='name')


In [46]:
(cities.dtypes, cities.geom_type.unique(), 
 cities.crs, cities.shape)

(geometry            object
 year        datetime64[ns]
 dtype: object,
 array(['Point'], dtype=object),
 {'init': 'epsg:4326'},
 (202, 2))

In [47]:
(retrieved_cities.dtypes, retrieved_cities.geom_type.unique(), 
 retrieved_cities.crs, retrieved_cities.shape)

(geometry            object
 year        datetime64[ns]
 dtype: object,
 array(['Point'], dtype=object),
 {'init': 'epsg:4326'},
 (202, 2))

# WIP: Tests

In [55]:
from geopandas.testing import assert_geodataframe_equal

In [56]:
assert_geodataframe_equal(countries, retrieved_countries, check_geom_type=True, check_less_precise=True)

# Spatial Query

In [ ]:
from sqlalchemy import Table, Column, String
from sqlalchemy.sql import select, text

table_name = 'city'
metadata = sqlalchemy.MetaData()
city_table = Table(table_name, metadata,
                   Column('name', String),
                     Column('geometry', Geometry('POINT', 4326))
                        )

In [ ]:
# s = select([text('city_geo')])
s = select([city_table])

In [ ]:
str(s)

In [ ]:
res = engine.execute(s)
for row in res:
    list(row)
    break
#     print(row['name'])

In [ ]:
from sqlalchemy import func

In [ ]:
table_name = 'country'
metadata = sqlalchemy.MetaData()
country_table = Table(table_name, metadata,
                   Column('name', String),
                     Column('geometry', Geometry('GEOMETRY', 4326))
                        )

In [ ]:
cities_wkte = cities.geometry.map(lambda x: WKTElement(x.wkt, srid=4326)
                                     ).astype(str)

In [ ]:
idx = countries.contains(cities.iloc[0].geometry)
countries.loc[idx]

In [ ]:
s = select([country_table], country_table.c.geometry.contains(
    cities_wkte.iloc[0]))

In [ ]:
str(s)

In [ ]:
res = engine.execute(s)
for row in res:
    list(row)[0]

In [ ]:
retrieved_gdf = gpd.read_postgis(s, engine, geom_col='geometry')

In [ ]:
retrieved_gdf

## Points within polygon :



In [ ]:
table_name = 'capital'
metadata = sqlalchemy.MetaData()
capital_table = Table(table_name, metadata,
                   Column('name', String),
                     Column('geometry', Geometry('POINT', 4326))
                        )

In [ ]:
table_name = 'country'
metadata = sqlalchemy.MetaData()
country_table = Table(table_name, metadata,
                   Column('name', String),
                     Column('geometry', Geometry('POLYGON', 4326))
                        )

In [ ]:
country_wkte = countries.geometry.map(lambda x: WKTElement(x.wkt, srid=4326)
                                     )

In [ ]:
my_shape_wkte = country_wkte.loc[countries.index == 'Italy'].iloc[0]

In [ ]:
my_shape_str = str(my_shape_wkte)


In [ ]:
s = select([capital_table], capital_table.c.geometry.ST_Intersects(
    my_shape_wkte))

In [ ]:
str(s)

In [ ]:
res = engine.execute(s)
for row in res:
    list(row)[0]

## Conclusion operations 
* Weird results with `contains` as Russia is already retrieved as containing vatican city
* Use intersects instead https://github.com/geoalchemy/geoalchemy2/issues/90

# WIP: Scale

Ingest 1e6 cities and retrieve the points in Italy

In [ ]:
def augment_gdf(gdf, factor=10):
    return gpd.GeoDataFrame(np.vstack([gdf.values] * factor), 
                            columns=gdf.columns, crs=gdf.crs)

In [ ]:
augmented_cities = augment_gdf(cities, 100000)
augmented_countries = augment_gdf(countries, 100000)

In [ ]:
augmented_cities.shape

In [ ]:
augmented_cities.to_postgis('capital', engine, if_exists='replace', chunksize=30000)

* Remote 

| time | documents | operation | 
| ---- | ------ | --- | 
| 36s | 2000 | ingestion | 
| 6min27 | 20000 | ingestion | 
| ?? | 200000 | ingestion | 
| 2s | 2,000,000 | query 30,000 points in 1 POLYGON  | 
| 2min16 | 2,000,000 | query ALL |
| 30s | 2,000,000 | query 440,000 points in 39 POLYGONS  | 


* On localhost


| time | documents | operation | 
| ---- | ------ | --- | 
| 1s| 20000 | ingestion | 
| 29s | 200000 | ingestion | 
| 7s | 200000 | query all  | 
| 1s | 200000 | query 3000 points in POLYGON  | 
| 5min10 | 2,000,000 | ingestion | 
| 1min40 | 2,000,000 | query ALL |
| 1s45 | 2,000,000 | query 30,000 points in POLYGON  | 
| Client Failed | 20,000,000 | ingestion  | 


In [ ]:
# Ingest 2000 points in 26s 

In [ ]:
%%time 
# Fetch all
retrieved_cities = gpd.read_postgis('capital', engine, geom_col='geometry')

In [ ]:
retrieved_cities.shape

In [ ]:
# Fetch in polygon
my_shape_wkte2 = country_wkte.loc[countries.name == 'Italy'].iloc[0]
my_shape_wkte3 = country_wkte.loc[countries.name == 'Spain'].iloc[0]

In [ ]:
expr = (capital_table.c.geometry.ST_Intersects(my_shape_wkte2) 
        | capital_table.c.geometry.ST_Intersects(my_shape_wkte3))

In [ ]:
from sqlalchemy import select
s = select([capital_table], 
          expr )

retrieved_points = gpd.read_postgis(s, engine,  geom_col='geometry')

In [ ]:
retrieved_points.name.value_counts()

In [ ]:
retrieved_points.shape

In [ ]:
# Simple within 
my_shape = countries.loc[countries.name == 'Italy'].iloc[0]
# my_shape = countries.loc[countries.name == 'Italy']

In [ ]:
retrieve_points2 = augmented_cities.within(my_shape.geometry)
retrieve_points2 = augmented_cities.loc[retrieve_points2]
retrieve_points2.shape

In [ ]:
retrieve_points2.sum()

## Query data from several polygons

In [ ]:
europe = countries.loc[countries.continent == 'Europe']
europe_wkte = europe.geometry.map(lambda x: WKTElement(x.wkt, srid=4326))


In [ ]:
from sqlalchemy import or_

In [ ]:
wktes = or_(capital_table.c.geometry.ST_Intersects(wkte) for wkte in europe_wkte)

In [ ]:
len(wktes)

In [ ]:
from sqlalchemy import select
s = select([capital_table], 
          wktes )

retrieved_points = gpd.read_postgis(s, engine,  geom_col='geometry')

In [ ]:
retrieved_points.name.value_counts().shape

In [ ]:
europe_wkte = europe.geometry.map(lambda x: WKTElement(x.wkt, srid=4326))

expr = (capital_table.c.geometry.ST_Intersects(my_shape_wkte2) 
        | capital_table.c.geometry.ST_Intersects(my_shape_wkte3))

# Read from POSTGIS

## Amazon

In [ ]:
db_json  = {"host": "dssg2017.csya4zsfb6y4.us-east-1.rds.amazonaws.com", 
 "port": 5432, 
 "database": "geohack",
 "user": "geohackstudent", 
"password": "geohackStudent2017"}

In [ ]:
import psycopg2

conn = psycopg2.connect(**db_json)

In [ ]:
hydrobas_ww = gpd.read_postgis(
    "SELECT * FROM hybas_na_lev00_v1c WHERE pfaf_4 = 7831", 
    conn, geom_col='polygongeom', crs={'init': u'epsg:4326'}, 
    coerce_float=False)

In [ ]:
hydrobas_ww.plot(column='pfaf_7', cmap='Paired', categorical=True, figsize=(14, 8))


In [ ]:
## Do aggregation and operations 